In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd


import import_ipynb
from Environments import Square_Crossroads

importing Jupyter notebook from Environments.ipynb
Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-16
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-11
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-11
A car tried to go outside the boundaries
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-16
A car tried to go outside the boundaries
-6
A car tried to go outside the boundaries
-6
A car tried to go outside the boundaries
-6
A car tried to go outside the boundaries
A car tried to go outside the boundaries
-11
-1
-1


In [2]:
n_times_steps = 3
seed = 10
dist_cars = 0.5

env = Square_Crossroads(n_times_steps, seed, dist_cars)
env.reset()

action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
total_reward = 0

print(env.states)


Environment reset with param
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [0, 5], 0, 0], 2: [[5, 10], [5, 0], 0, 0], 3: [[10, 5], [5, 0], 0, 0]}


In [3]:
state_tensor = torch.zeros([4,2], dtype=torch.float32)
action_tensor = torch.zeros([4,2], dtype=torch.float32)

In [4]:
class Actor_Critic(nn.Module):
    def __init__(self, state_tensor, hidden_size_actor, hidden_size_critic):
        super(Actor_Critic, self).__init__()
      
        self.num_inputs = state_tensor.shape[1]
        
        
        self.critic_linear1 = nn.Linear(self.num_inputs, hidden_size_critic)
        self.critic_linear2 = nn.Linear(hidden_size_critic, 1)
        
        self.actor_linear1 = nn.Linear(self.num_inputs, hidden_size_actor)
        self.actor_linear2 = nn.Linear(hidden_size_actor, hidden_size_actor)
        self.alpha = nn.Linear(hidden_size_actor, 1)
        self.beta = nn.Linear(hidden_size_actor, 1)
        

    def forward(self, state_tensor):

        value = F.relu(self.critic_linear1(state_tensor))
        value = self.critic_linear2(value)
        
        policy_dist = F.relu(self.actor_linear1(state_tensor))
        policy_dist = F.relu(self.actor_linear2(policy_dist))

        policy_dist_alpha = torch.clamp(self.alpha(policy_dist), 0.0001, 0.999)
        policy_dist_beta = torch.clamp(self.beta(policy_dist), 0.0001, 0.999)
        
        
        return value, policy_dist_alpha, policy_dist_beta

In [5]:
ac = Actor_Critic(state_tensor, 20, 400)

In [6]:
ac

Actor_Critic(
  (critic_linear1): Linear(in_features=2, out_features=400, bias=True)
  (critic_linear2): Linear(in_features=400, out_features=1, bias=True)
  (actor_linear1): Linear(in_features=2, out_features=20, bias=True)
  (actor_linear2): Linear(in_features=20, out_features=20, bias=True)
  (alpha): Linear(in_features=20, out_features=1, bias=True)
  (beta): Linear(in_features=20, out_features=1, bias=True)
)

In [7]:
value, policy_dist_alpha, policy_dist_beta = ac(state_tensor)

In [8]:
policy_dist_alpha

tensor([[0.0366],
        [0.0366],
        [0.0366],
        [0.0366]], grad_fn=<ClampBackward>)

In [9]:
policy_dist_beta

tensor([[0.0208],
        [0.0208],
        [0.0208],
        [0.0208]], grad_fn=<ClampBackward>)

In [10]:
value

tensor([[0.0686],
        [0.0686],
        [0.0686],
        [0.0686]], grad_fn=<AddmmBackward>)

In [11]:
a = ac.forward(state_tensor)

In [12]:
a

(tensor([[0.0686],
         [0.0686],
         [0.0686],
         [0.0686]], grad_fn=<AddmmBackward>), tensor([[0.0366],
         [0.0366],
         [0.0366],
         [0.0366]], grad_fn=<ClampBackward>), tensor([[0.0208],
         [0.0208],
         [0.0208],
         [0.0208]], grad_fn=<ClampBackward>))